<a href="https://colab.research.google.com/github/Benedictakel/Rule-Based-Chatbot-with-Seq2Seq-Small-Conversation-/blob/main/Rule_Based_Chatbot_with_Seq2Seq_(Small_Conversation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [3]:
eng_sentences = ["hello", "how are you", "good morning", "thank you"]
fra_sentences = ["bonjour", "comment ça va", "bon matin", "merci"]


In [4]:
tokenizer_en = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer_fr = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer_en.fit_on_texts(eng_sentences)
tokenizer_fr.fit_on_texts(fra_sentences)

seq_en = tokenizer_en.texts_to_sequences(eng_sentences)
seq_fr = tokenizer_fr.texts_to_sequences(fra_sentences)

max_en = max(len(s) for s in seq_en)
max_fr = max(len(s) for s in seq_fr)

seq_en = tf.keras.preprocessing.sequence.pad_sequences(seq_en, maxlen=max_en, padding='post')
seq_fr = tf.keras.preprocessing.sequence.pad_sequences(seq_fr, maxlen=max_fr, padding='post')

In [7]:
latent_dim=32
# Encoder
enc_inputs = layers.Input(shape=(max_en,))
enc_emb = layers.Embedding(len(tokenizer_en.word_index)+1, latent_dim)(enc_inputs)
_, state_h, state_c = layers.LSTM(latent_dim, return_state=True)(enc_emb)
enc_states = [state_h, state_c]
# Decoder
dec_inputs = layers.Input(shape=(max_fr,))
dec_emb = layers.Embedding(len(tokenizer_fr.word_index)+1, latent_dim)(dec_inputs)
dec_lstm = layers.LSTM(latent_dim, return_sequences=True, return_state=True)
dec_outputs, _, _ = dec_lstm(dec_emb, initial_state=enc_states)
dec_dense = layers.Dense(len(tokenizer_fr.word_index)+1, activation='softmax')
dec_outputs = dec_dense(dec_outputs)

model = tf.keras.Model([enc_inputs, dec_inputs], dec_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
model.fit([seq_en, seq_fr], seq_fr.reshape(seq_fr.shape[0], -1, 1), epochs=500, verbose=0)


In [9]:
model.save('seq2seq_small.keras')
print("Saved seq2seq_small/")

Saved seq2seq_small/
